# Core functionality

> Basic processing

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *  # noqa: F403
import os


In [ ]:
#| export
import platform
import os

import typing
import json
import logging
import datetime
import numpy as np
import pandas as pd

from functools import lru_cache
from fastcore.basics import patch
from corebridge import __version__


## Logging

In [ ]:
#| exports

def init_console_logging(name='root', level=logging.INFO, timestamp=True):
    '''Setup none-blocking stream handler for sending loggin to the console.'''

    # Only if no handlers defined.
    if not logging.getLogger(name).handlers:

        logger = logging.getLogger(name)
        logger.setLevel(level)

        console = logging.StreamHandler()
        console.setLevel(level)

        # set a format which is simpler for console use
        if timestamp:
            formatter = logging.Formatter("%(asctime)s %(levelname)s\t%(process)d\t%(name)s\t%(filename)s\t%(lineno)d\t%(message)s", datefmt='%Y-%m-%dT%H:%M:%S%z')
        else:
            formatter = logging.Formatter("%(levelname)s\t%(process)d\t%(name)s\t%(filename)s\t%(lineno)d\t%(message)s")
            
        #formatter = logging.Formatter("%(asctime)s %(levelname)s\t%(process)d\t%(name)s\t%(filename)s\t%(lineno)d\t%(message)s", datefmt='%Y-%m-%dT%H:%M:%S%z')

        # tell the handler to use this format
        console.setFormatter(formatter)

        # add the handler to the root logger
        logger.addHandler(console)
        return logger
    else:
        logging.getLogger(name).info(f'There already is a logger installed for {name}.')


In [ ]:
init_console_logging()

<RootLogger root (INFO)>

In [ ]:
#| export
logging.basicConfig(
    format="%(asctime)s \t%(levelname)s\t%(funcName)s\t%(lineno)d\t%(message)s",
    datefmt="%Y-%m-%dT%H:%M:%S%z",
    level=logging.DEBUG
)

syslog = logging.getLogger(__name__)

try:
    logging.getLogger(__name__).info(f"Loading {__name__} from {__file__}")
except:  # noqa: E722
    pass



In [ ]:
syslog.info('Test')

2025-11-03T12:54:03+0100 INFO	1060	__main__	130201418.py	1	Test


## Machine information

In [ ]:
#| exporti
def get_machine_info():
    # Get OS details
    os_name = os.name  # 'posix', 'nt', or 'os2'
    system_name = platform.system()  # e.g., 'Windows', 'Linux', 'Darwin'
    release = platform.release()  # OS release version
    version = platform.version()  # OS version
    architecture = platform.architecture()  # e.g., ('64bit', 'ELF')
    machine = platform.machine()  # e.g., 'x86_64', 'AMD64'
    processor = platform.processor()  # Processor type
    node_name = platform.node()  # Network name of the machine

    # Display the gathered information
    return {
        "OS Name": os_name,
        "System Name": system_name,
        "Release": release,
        "Version": version,
        "Architecture": architecture,
        "Machine": machine,
        "Processor": processor,
        "Node Name": node_name,
    }



In [ ]:
get_machine_info()

{'OS Name': 'posix',
 'System Name': 'Linux',
 'Release': '5.14.21-150500.55.88-default',
 'Version': '#1 SMP PREEMPT_DYNAMIC Fri Dec 6 16:43:20 UTC 2024 (e102a89)',
 'Architecture': ('64bit', 'ELF'),
 'Machine': 'x86_64',
 'Processor': 'x86_64',
 'Node Name': 'bouwdoosje'}

## Strings

In [ ]:
#| exports
@lru_cache(128)
def snake_case_to_camel_case(snake_case:str) -> str:
    splittext = snake_case.split('_')
    return ''.join([x.capitalize() if n > 0 else x for x,n in zip(splittext, range(len(splittext)))])


## Numpy data in JSON

In [ ]:
#| export
#| echo: false
class NumpyEncoder(json.JSONEncoder):
    """ Custom encoder for numpy data types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64, np.int64)): # type: ignore

            return int(obj)

        elif isinstance(obj, (np.float16, np.float32, np.float64)): # type: ignore
            return float(obj)
        
        elif isinstance(obj, (np.complex64, np.complex128)): # type: ignore
            return {'real': obj.real, 'imag': obj.imag}
        
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
    
        elif isinstance(obj, (np.bool_)):
            return bool(obj)

        elif isinstance(obj, (np.void)): 
            return None
        
        elif isinstance(obj, np.datetime64):
            return str(obj)

        elif isinstance(obj, datetime.datetime):
            return obj.isoformat()

        return json.JSONEncoder.default(self, obj)


## Timeseries dataframes

Timeseries data is a cornerstone of our data manipulation and most processing is on them

### `set_time_index_zone`

Processing may depend on proper timezone awareness, this utility to set the timezone on a datetime index

In [ ]:
#| export

def set_time_index_zone(
        df:pd.DataFrame,    # Dataframe to set or convert the timeindex on
        timezone            # Timezone to set
    ) :
    """
    Sets the time zone of the index of a pandas DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame whose index time zone is to be set.
        timezone (str): The desired time zone.

    Returns:
        pd.DataFrame: The modified DataFrame with its index time zone set 
        to the specified time zone.

    Raises:
        None

    Examples:
        >>> df = pd.DataFrame({'A': [1, 2, 3]}, index=pd.DatetimeIndex(['2022-01-01', '2022-01-02', '2022-01-03']))
        >>> set_time_index_zone(df, 'Europe/Berlin')
                     A
        2022-01-01  1
        2022-01-02  2
        2022-01-03  3
        DatetimeIndex: 3 entries, 2022-01-01 01:00:00+01:00 to 2022-01-03 01:00:00+01:00
    """
    
    if isinstance(df.index, pd.DatetimeIndex):
        df.index.name = 'time'
        if not hasattr(df.index, 'tz')  or not df.index.tz or not df.index.tz:
            df.index = df.index.tz_localize('UTC').tz_convert(timezone)
        elif str(df.index.tz) != timezone:
            df.index = df.index.tz_convert(timezone)

    return df


#### Example

In [ ]:
df = pd.DataFrame({'A': [1, 2, 3]}, index=pd.DatetimeIndex(['2022-01-01', '2022-01-02', '2022-01-03']))
set_time_index_zone(df, 'Europe/Berlin')
df.index

DatetimeIndex(['2022-01-01 01:00:00+01:00', '2022-01-02 01:00:00+01:00',
               '2022-01-03 01:00:00+01:00'],
              dtype='datetime64[ns, Europe/Berlin]', name='time', freq=None)

### timeseries_dataframe

Converts Pandas dataframes and series, Numpy array's and recarrays or a dictionary of 
individual timeseries into a Pandas dataframe with one datetime index. With all arrays 
dataframes and series it is assumed that the first column contains the timestamps.

In [ ]:
#| export

def timeseries_dataframe(
        data:typing.Union[pd.DataFrame, pd.Series, dict, np.ndarray, np.recarray], 
        timezone='UTC', 
        columnnames=None):
    
    """Convert various tabular data formats to timeseries DataFrame

    Args:
        data (Union[pd.DataFrame, pd.Series, dict, np.ndarray, np.recarray]): The input data to be converted.
        timezone (str, optional): The timezone to set for the index of the DataFrame. Defaults to 'UTC'.
        columnnames (Optional[List[str]]): The column names to use for the DataFrame. Defaults to None.

    Returns:
        pd.DataFrame: The converted timeseries DataFrame with the index set to the specified timezone.
    """

    if isinstance(data, pd.DataFrame):
        df = data.copy()
        df.index = pd.DatetimeIndex(df.index).round('ms')

    elif isinstance(data, pd.Series):
        df = pd.DataFrame(data)
        df.index = pd.DatetimeIndex(df.index).round('ms')

    elif isinstance(data, dict):
        # assume a dict/mapping of individual arrays representing timeseries 
        df = pd.DataFrame({
            C:pd.Series(data=A[:,1], index=pd.DatetimeIndex(A[:,0]*1e9)) if isinstance(A, np.ndarray) else A
            for C,A in data.items()
        })

    elif data.dtype.names is not None:
        # structured or recarray, we use column names from the recarray
        df = pd.DataFrame(
            data=data.view(dtype=np.float64).reshape(data.shape[0],len(data.dtype))[:,range(1,len(data.dtype))],
            index=pd.DatetimeIndex(data.view(dtype=np.float64).reshape(data.shape[0],len(data.dtype))[:,0] * 1e9),
            columns=data.dtype.names[1:]
        )

    else:
        if data.shape[0] > 0:
            # column names, either 'value' if there is only one column, or
            # value_0, value_1 .... value_nn when more the one column is present
            if data.shape[1]>2:
                columns=[f"value_{str(i+1)}" for i in range(data.shape[1]-1)] if not columnnames else [f"{str(i)}" for i in columnnames[1:]]
            else:
                columns=['value']

            df = pd.DataFrame(
                data=data[:, 1:],
                index=pd.DatetimeIndex(data[:,0]*1e9),
                columns=columns
            )
        else:
            return pd.DataFrame()

    return set_time_index_zone(df, timezone)

### `timeseries_dataframe_from_datadict`

In [ ]:
#| exports
def timeseries_dataframe_from_datadict(
        data:dict, 
        timecolumns=None,
        recordformat='records'):
        
    """
    Converts a data dict into a pandas DataFrame based on the specified record format. 
    Parameters:
        - data: A dictionary containing the data to convert.
        - timecolumns: A list of column names to be treated as time columns.
        - recordformat: A string specifying the format of the data records ('records', 'table', 'split', 'index', 'tight').
    Returns:
        - df: A pandas DataFrame with a DatetimeIndex representing the converted data.
    """

    orient = recordformat.lower()
    assert orient in ['records', 'table', 'split', 'index', 'tight']
    assert timecolumns, 'No time columns specified'

    if orient == 'records':
        # data is a structured ndarray, sequence of tuples or dicts, or DataFrame
        df = pd.DataFrame.from_records(data)
        time_columns_in_df = [C for C in df.columns if C in timecolumns]
        if not  time_columns_in_df:
            #syslog.error(f"No  column in records {df.columns} matches specification in time columns {timecolumns}, assuming first column is time")
            time_column = df.columns[0]
        else:
            time_column = time_columns_in_df[0]

    elif orient == 'table':
        # data is in pandas table format
        time_column = data['schema']['primaryKey'][0]
        df = pd.DataFrame.from_dict(data['data']).set_index(data['schema']['primaryKey'])
        df.index.name = 'time'
    else:
        # data  is formatted according to 'orient' parameter (pandas)
        df = pd.DataFrame.from_dict(data, orient=orient) # type: ignore
        time_column = df.index.name


    df.columns = list(df.columns)
    df[time_column] = pd.to_datetime(df[time_column],utc=True,format='ISO8601')
    df.set_index(time_column, inplace=True)
    df.index = pd.DatetimeIndex(df.index).round('ms')
    
    df.index.name = 'time'

    return df


In [ ]:
df = timeseries_dataframe_from_datadict([
      {
         "time":"2023-05-04T10:04:49.000Z",
         "value":16.72
      },
      {
         "time":"2023-05-04T10:24:51.000Z",
         "value":16.65
      },
      {
         "time":"2023-05-04T10:44:53.000Z",
         "value":16.55
      }
   ], timecolumns=['time'])
df

,value
time,
2023-05-04 10:04:49+00:00,16.72
2023-05-04 10:24:51+00:00,16.65
2023-05-04 10:44:53+00:00,16.55


In [ ]:
df.index

DatetimeIndex(['2023-05-04 10:04:49+00:00', '2023-05-04 10:24:51+00:00',
               '2023-05-04 10:44:53+00:00'],
              dtype='datetime64[ns, UTC]', name='time', freq=None)

In [ ]:
df = set_time_index_zone( timeseries_dataframe_from_datadict([
      {
         "time":"2023-05-04T10:04:49",
         "value":16.72
      },
      {
         "time":"2023-05-04T10:24:51",
         "value":16.65
      },
      {
         "time":"2023-05-04T10:44:53",
         "value":16.55
      }
   ], timecolumns=['time']), timezone='Europe/Amsterdam')
df

,value
time,
2023-05-04 12:04:49+02:00,16.72
2023-05-04 12:24:51+02:00,16.65
2023-05-04 12:44:53+02:00,16.55


In [ ]:
df.index

DatetimeIndex(['2023-05-04 12:04:49+02:00', '2023-05-04 12:24:51+02:00',
               '2023-05-04 12:44:53+02:00'],
              dtype='datetime64[ns, Europe/Amsterdam]', name='time', freq=None)

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()